This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
import collections
import numpy as np

Note: API's can change a bit with each version, for this exercise it is reccomended to use the "V3" quandl api at `https://www.quandl.com/api/v3/`

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-01-03&api_key=' + API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
AFX_OneDay = r.json()
for key, value in AFX_OneDay.items():
    print(key + ':', value)

dataset: {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-12-01T14:48:09.907Z', 'newest_available_date': '2020-12-01', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2017-01-01', 'end_date': '2017-01-03', 'data': [['2017-01-03', 35.9, 35.93, 35.34, 35.48, None, 70618.0, 2515473.0, None, None, None], ['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}


The json file downloaded from Quandl API is a nested dictionary with the primary key "dataset" and a value, which in itself is a dictionary. The nested dictionary conatians several keys and value pairs. The two key values we are interset are the "column_names" and "data" keys.

In [5]:
print(value.keys())

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])


In [6]:
#Q.1 Now, call the Quandl API and pull out the 2017 whole year value for AFX_X
req = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY)

In [7]:
#Q.2 Convert the requested json file into a dictionary
AFX_2017 = req.json()
type(AFX_2017)

dict

In [8]:
#[key_2017, value_2017 for key_2017, value_2017 in AFX_2017.items()]
for key_2017, value_2017 in AFX_2017.items():
    pass
    #print(key_2017 + ':', value_2017)

In [9]:
column_names = value_2017['column_names']
print(column_names )

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [10]:
data = value_2017['data']
#print(data)

From the above inspection we know that the two important keys are the column_names and data keys. Now we will form a new dictionary that pairs the name of each items in column_names key to the items in the data key

In [11]:
# Create a new dictionary that only contains the data and the column names 
AFX_2017_select = collections.defaultdict(list)
for i in range(len(data)):
    for key, value in zip(column_names,data[i]):
        if key not in AFX_2017_select:
            AFX_2017_select[key] = [value] # the square bracket tells the value will be a list instead of just a string
        else:
            AFX_2017_select[key].append(value)

In [12]:
#Q.3 Calculate what the highest and lowest opening prices were for the stock in this period.
AFX_Opening = AFX_2017_select['Open']
AFX_Opening = [i if i is not None else np.NaN for i in AFX_Opening]
AFX_Lowest_Opening = min(AFX_Opening)
AFX_Highest_Opening = max(AFX_Opening)
print('The highest and lowest opening price for AFX in 2017 are ${} and ${} respectively'.format(AFX_Highest_Opening,AFX_Lowest_Opening))

The highest and lowest opening price for AFX in 2017 are $53.11 and $34.0 respectively


In [13]:
#Q.4 What was the largest change in any one day (based on High and Low price)?
AFX_High = AFX_2017_select['High']
AFX_High = [i if i is not None else np.NaN for i in AFX_High]

AFX_Low = AFX_2017_select['Low']
AFX_Low = [i if i is not None else np.NaN for i in AFX_Low]

AFX_Daily_Change = np.array(AFX_High) - np.array(AFX_Low)
print('The largest price change in a single day is ${:.2f}'.format(max(AFX_Daily_Change)))

The largest price change in a single day is $2.81


In [14]:
#Q.5 What was the largest change between any two days (based on Closing Price)?
AFX_2017_select['Diff_2Day_prior'] = []
num = len(AFX_2017_select['Close'])

for i in range(num):
    try:
        AFX_2017_select['Diff_2Day_prior'].append(round(AFX_2017_select['Close'][i+1] - AFX_2017_select['Close'][i],2))
    except IndexError:
        AFX_2017_select['Diff_2Day_prior'].append(round(AFX_2017_select['Close'][i-1] - AFX_2017_select['Close'][i],2))
        
print('The largest price change between any two days is ${}'.format(max((abs(x) for x in AFX_2017_select['Diff_2Day_prior'] if x is not None))))       

print('\r')

#OR using the numpy function np.diff()

AFX_Close = AFX_2017_select['Close']
AFX_Close = [i if i is not None else np.NaN for i in AFX_Close]

AFX_2Day_Diff = np.diff(np.array(AFX_Close))
print('The largest price change between any two days is ${}'.format(round(abs(max(AFX_2Day_Diff)),2)))

The largest price change between any two days is $2.56

The largest price change between any two days is $2.56


In [15]:
#Q.6 What was the average daily trading volume during this year?
AFX_Traded_Vol = np.array(AFX_2017_select['Traded Volume'])

Traded_Vol_sorted = sorted(AFX_Traded_Vol)
num = len(Traded_Vol_sorted)
mean = sum(AFX_Traded_Vol)/num

print('The mean daily trading volume during for 2017 is {:.2f}'.format(mean))

print("\r")

#OR using numpy mean function
print('The mean daily trading volume during for 2017 is {:.2f}'.format(np.mean(AFX_Traded_Vol)))

The mean daily trading volume during for 2017 is 89124.34

The mean daily trading volume during for 2017 is 89124.34


In [16]:
#Q.7 What was the median trading volume during this year.

if num % 2 == 0:
    median = (Traded_Vol_sorted[num//2] + Traded_Vol_sorted[num//2 -1]) / 2
else:
    median = Traded_Vol_sorted[num//2]

print('The median daily trading volume during for 2017 is {:.2f}'.format(median))

print("\r")

#OR using numpy median function
print('The median daily trading volume during for 2017 is {:.2f}'.format(np.median(AFX_Traded_Vol)))

The median daily trading volume during for 2017 is 76286.00

The median daily trading volume during for 2017 is 76286.00


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)